In [4]:
import scanpy as sc, warnings, numpy as np, pandas as pd
sc.settings.set_figure_params(dpi=80, facecolor="white", frameon=False)
from tqdm.auto import tqdm
sc.settings.verbosity = 3
warnings.filterwarnings("ignore")

### save sepreate datasets

In [3]:
%%time

adata = sc.read_h5ad("Kidney_Combined_cellbender_v2.h5ad")
columns_to_drop = [col for col in adata.obs.columns if "RNA_snn" in col or "_prob" in col]
adata.obs.drop(columns=columns_to_drop, inplace=True)
adata

CPU times: user 4.84 s, sys: 9.28 s, total: 14.1 s
Wall time: 25.2 s


AnnData object with n_obs × n_vars = 2340396 × 36601
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name'

In [4]:
sc.pp.filter_cells(adata, min_genes=1)
sc.pp.filter_genes(adata, min_counts=1)

filtered out 364830 cells that have less than 1 genes expressed
filtered out 2464 genes that are detected in less than 1 counts


In [5]:
snrna_dataset = [
    'Kidney_Muto2', 
    'Kidney_Muto',
    'Kidney_Wilson2',
    'Kidney_Wilson',
    'Kidney_Lake', 
    'Kidney_Wu'
]

for item in snrna_dataset:
    assert item in adata.obs["Project_ID"].values


adata.obs["modality"] = "scrna"
mask = [item in snrna_dataset for item in adata.obs["Project_ID"].values]
adata.obs.loc[mask, "modality"] = "snrna"

In [7]:
%%time
adata[adata.obs["modality"] == "snrna"].write("Kidney_Combined_sn.h5ad", compression="gzip")

/home/wangh256/anaconda3/envs/scIntegrater/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


In [8]:
%%time
adata[adata.obs["modality"] == "scrna"].write("Kidney_Combined_sc.h5ad", compression="gzip")

/home/wangh256/anaconda3/envs/scIntegrater/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


### Check runned integration

In [5]:
adata = sc.read_h5ad("Kidney_Combined_sn_hvg5k.h5ad")
adata

AnnData object with n_obs × n_vars = 362175 × 5000
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name', 'n_genes', 'modality', '_scvi_batch', '_scvi_labels', 'conditions_combined'
    var: 'n_counts', 'highly_variable', 'highly_variable

In [6]:
adata.obsm

AxisArrays with keys: Harmony, Scanorama, X_bbknn, X_pca, X_scANVI, X_scVI, X_tsne, X_umap, scPoli

In [8]:
del adata.obsm["X_scANVI"]
del adata.obsm["scPoli"]

In [ ]:
## saved labeled subset
